In [22]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import SimpleImputer

In [23]:
workout_df = pd.read_csv('/content/megaGymDataset.csv')
workout_df.head()

,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN


In [24]:
workout_df.drop(columns=['Rating', 'RatingDesc', 'Unnamed: 0'], inplace=True)

missing_values_count = workout_df.isnull().sum()
print("Missing values in each column:\n", missing_values_count)

Missing values in each column:
 Title           0
Desc         1550
Type            0
BodyPart        0
Equipment      32
Level           0
dtype: int64


In [25]:
count_exercises = workout_df['BodyPart'].value_counts().reset_index(name='Count')
count_exercises.columns = ['BodyPart', 'Count']
fig = px.bar(count_exercises.sort_values(by='Count', ascending=False), x='BodyPart', y='Count', title='Count of Exercises', labels={'BodyPart': 'Exercise Type', 'Count': 'Count'})
fig.show()

In [26]:
one_hot = OneHotEncoder(sparse_output=False)
one_hot_df = pd.DataFrame(one_hot.fit_transform(workout_df[['Type', 'BodyPart']]),
                           columns=one_hot.get_feature_names_out(['Type', 'BodyPart']))

workout_df = workout_df.join(one_hot_df)

workout_df['Equipment_Required'] = workout_df['Equipment'].apply(lambda x: 0 if x == 'None' else 1)
workout_df['Level_Score'] = workout_df['Level'].map({'Beginner': 1, 'Intermediate': 2, 'Advanced': 3})

In [27]:
features = workout_df.drop(columns=['Title', 'Desc', 'Equipment', 'Level'])
X = features.select_dtypes(include=['float64', 'int64']).values
y = workout_df['Title']

imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
knn = NearestNeighbors(n_neighbors=5)
knn.fit(X_train)

NearestNeighbors()

In [30]:
def recommend_workouts(user_input):
    user_df = pd.DataFrame([user_input])
    all_columns = features.select_dtypes(include=['float64', 'int64']).columns
    user_input_transformed = user_df.reindex(columns=all_columns, fill_value=0)
    user_input_transformed = imputer.transform(user_input_transformed)

    distances, indices = knn.kneighbors(user_input_transformed)
    recommended_titles = y_train.iloc[indices[0]].values
    return recommended_titles

In [31]:
user_input = {
    'Equipment_Required': 1,
    'Level_Score': 2,
    'Type_Strength': 1,
    'Type_Cardio': 0,
    'BodyPart_Abdominals': 1,
    'BodyPart_Legs': 0,
    'BodyPart_Arms': 0,
    'BodyPart_Back': 0,
    'BodyPart_Chest': 0,
}

recommended_workouts = recommend_workouts(user_input)
print("Recommended Workouts:")
for title in recommended_workouts:
    print(title)

Recommended Workouts:
Bench crunch
Exercise ball knee roll-in
Suicides
Alligator Crawl
Exercise Ball Cable Crunch - Gethin Variation


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SimpleImputer was fitted without feature names



In [32]:
pip install fuzzywuzzy

In [33]:
# Importing Required Library
from fuzzywuzzy import fuzz

# Function to Evaluate Recommendations
def evaluate_recommendations(recommended, expected):
    recommended_set = set(recommended)
    hits = 0

    for expected_title in expected:
        for recommended_title in recommended:
            if fuzz.partial_ratio(expected_title.lower(), recommended_title.lower()) > 80:  # Adjust threshold as needed
                hits += 1
                break  # Only count one hit per expected title

    precision = hits / len(recommended_set) if recommended_set else 0
    recall = hits / len(expected) if expected else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# Define Expected Titles Based on Your Dataset
expected_titles = ["Bench crunch", "Banded crunch"]  # Adjust as needed

# Get Recommended Workouts
recommended_workouts = recommend_workouts(user_input)
print("Recommended Workouts:", recommended_workouts)

# Evaluate the Recommendations
precision, recall, f1 = evaluate_recommendations(recommended_workouts, expected_titles)

# Print Evaluation Metrics
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Recommended Workouts: ['Bench crunch' 'Exercise ball knee roll-in' 'Suicides' 'Alligator Crawl'
 'Exercise Ball Cable Crunch - Gethin Variation']
Precision: 0.20
Recall: 0.50
F1 Score: 0.29


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SimpleImputer was fitted without feature names

